<a href="https://colab.research.google.com/github/detektor777/colab_list_video/blob/main/flavr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture

import os
import subprocess
import importlib.util
from google.colab import files
import torch

if 'downloaded_model_type' not in globals():
    downloaded_model_type = None

model_type = "2x"  #@param ["2x", "4x", "8x"]

print(f"Current PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

print("Installing FLAVR dependencies...")

print("Installing numpy...")
result_numpy = subprocess.run(["pip", "install", "numpy==1.23.5"], capture_output=True, text=True)
if result_numpy.returncode != 0:
    print("Error installing numpy:")
    print(result_numpy.stderr)

print("Installing opencv-python...")
result_opencv = subprocess.run(["pip", "install", "opencv-python==4.8.0.76"], capture_output=True, text=True)
if result_opencv.returncode != 0:
    print("Error installing opencv-python:")
    print(result_opencv.stderr)

print("Installing PyAV...")
result_av = subprocess.run(["pip", "install", "av==10.0.0"], capture_output=True, text=True)
if result_av.returncode != 0:
    print("Error installing PyAV:")
    print(result_av.stderr)

if importlib.util.find_spec("av") is not None:
    import av
    print(f"PyAV successfully installed, version: {av.__version__}")
else:
    print("Error: PyAV is not installed after installation attempt.")

if not os.path.exists("/content/FLAVR"):
    print("Cloning FLAVR repository...")
    result_git = subprocess.run(["git", "clone", "https://github.com/tarun005/FLAVR.git", "/content/FLAVR"], capture_output=True, text=True)
    if result_git.returncode != 0:
        print("Error cloning repository:")
        print(result_git.stderr)
    else:
        print("Repository successfully cloned.")
else:
    print("FLAVR repository already exists.")

model_urls = {
    "2x": "1IZe-39ZuXy3OheGJC-fT3shZocGYuNdH",
    "4x": "1xoZqWJdIOjSaE2DtH4ifXKlRwFySm5Gq",
    "8x": "1DlXgNANDGLZEYOCMvQ5T1cAqkW90FiPt"
}
model_path = f"/content/FLAVR_{model_type}.pth"

if os.path.exists(model_path) and downloaded_model_type == model_type:
    print(f"Model {model_type} is already downloaded and matches the selected type: {model_path}")
elif os.path.exists(model_path) and downloaded_model_type != model_type:
    print(f"Model {downloaded_model_type} already exists, but {model_type} is selected. Replacing model...")
    os.remove(model_path)
    download_required = True
else:
    print(f"Model {model_type} is missing. Download required.")
    download_required = True

if download_required:
    print(f"Downloading FLAVR model {model_type}...")
    subprocess.run(["pip", "install", "gdown"], capture_output=True, text=True)
    result_gdown = subprocess.run(["gdown", f"https://drive.google.com/uc?id={model_urls[model_type]}", "-O", model_path], capture_output=True, text=True)

    if result_gdown.returncode == 0 and os.path.exists(model_path):
        global downloaded_model_type
        downloaded_model_type = model_type
        print(f"Model {model_type} successfully downloaded to {model_path}")
    else:
        print(f"Error: Failed to automatically download model {model_type}.")
        print(result_gdown.stderr)
        print("Please download the model manually:")
        print(f"1. Use the link: https://drive.google.com/file/d/{model_urls[model_type]}/view")
        print(f"2. Download the file and upload it below with the name FLAVR_{model_type}.pth")
        uploaded = files.upload()
        for uploaded_file_name, _ in uploaded.items():
            if uploaded_file_name.endswith(".pth"):
                os.rename(uploaded_file_name, model_path)
                downloaded_model_type = model_type
                print(f"Model saved as {model_path}")
                break
        else:
            print("Error: Uploaded file is not a .pth model.")



In [ ]:
#@title ##**Select Video File** { display-mode: "form" }
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from google.colab import files
from google.colab import drive

upload_option = "Upload from PC"  #@param ["Upload from PC", "Load from Google Drive Root", "Load from Google Drive"]

file_name = None
last_selected_button = None

def reset_button_colors(buttons):
    for btn in buttons:
        btn.style.button_color = None

if upload_option == "Upload from PC":
    print("Please upload a video file.")
    uploaded = files.upload()
    if uploaded:
        file_name = list(uploaded.keys())[0]
    else:
        print("No file uploaded.")
        file_name = None

elif upload_option == "Load from Google Drive Root":
    drive.mount('/content/drive')
    root_dir = '/content/drive/MyDrive/'

    video_extensions = ['.mp4', '.mkv', '.avi', '.mov']
    files_list = []

    for f in os.listdir(root_dir):
        if os.path.isfile(os.path.join(root_dir, f)) and os.path.splitext(f)[1].lower() in video_extensions:
            files_list.append(f)

    if not files_list:
        print("No video files found in Google Drive root.")
        file_name = None
    else:
        print("Select a video file from Google Drive root:")

        output = widgets.Output()
        buttons = []

        def on_button_clicked(b):
            global file_name, last_selected_button
            with output:
                clear_output()
                reset_button_colors(buttons)
                selected_file = b.description
                file_name = os.path.join(root_dir, selected_file)

                if file_name and os.path.exists(file_name):
                    b.style.button_color = 'green'
                else:
                    b.style.button_color = 'red'

                last_selected_button = b
                print(f"Selected file: {file_name if file_name else 'None'}")

        for file in files_list:
            button = widgets.Button(description=file, layout=widgets.Layout(width='500px', overflow='hidden', text_overflow='ellipsis'))
            button.on_click(on_button_clicked)
            buttons.append(button)

        display(widgets.VBox(buttons), output)

elif upload_option == "Load from Google Drive":
    drive.mount('/content/drive')
    root_dir = '/content/drive/MyDrive/'

    video_extensions = ['.mp4', '.mkv', '.avi', '.mov']
    files_list = []

    for dirpath, _, filenames in os.walk(root_dir):
        for f in filenames:
            if os.path.splitext(f)[1].lower() in video_extensions:
                relative_path = os.path.relpath(os.path.join(dirpath, f), root_dir)
                files_list.append(relative_path)

    if not files_list:
        print("No video files found in Google Drive or its subfolders.")
        file_name = None
    else:
        print("Select a video file from Google Drive (including subfolders):")

        output = widgets.Output()
        buttons = []

        def on_button_clicked(b):
            global file_name, last_selected_button
            with output:
                clear_output()
                reset_button_colors(buttons)
                selected_file = b.description
                file_name = os.path.join(root_dir, selected_file)

                if file_name and os.path.exists(file_name):
                    b.style.button_color = 'green'
                else:
                    b.style.button_color = 'red'

                last_selected_button = b
                print(f"Selected file: {file_name if file_name else 'None'}")

        for file in files_list:
            button = widgets.Button(description=file, layout=widgets.Layout(width='500px', overflow='hidden', text_overflow='ellipsis'))
            button.on_click(on_button_clicked)
            buttons.append(button)

        display(widgets.VBox(buttons), output)

if file_name:
    print(f"Video file path set to: {file_name}")
else:
    print("Video file path not set. Please select a file.")

In [ ]:
#@title ##**Config** { display-mode: "form" }
import os
from google.colab import files
import shutil
from google.colab import drive

interpolation_factor = 2  #@param {type:"slider", min:2, max:8, step:2}  # Умножение FPS: 2x, 4x, 8x
output_folder = "root" #@param ["google_drive","root"]
segment_duration = 4  #@param {type:"slider", min:1, max:60, step:1}  # Длительность сегмента в секундах

if output_folder == "google_drive":
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')


In [ ]:
#@title ##**Run** { display-mode: "form" }
import subprocess
import os
import shutil
from google.colab import files
import cv2
import gc
import sys
import time
import logging

if 'file_name' not in globals() or file_name is None or not os.path.exists(file_name):
    print("Error: Video file is not selected or does not exist. Run the 'Select Video File' cell.")
else:
    print(f"Increasing FPS for video: {file_name}")
    print(f"Input file size: {os.path.getsize(file_name) / (1024*1024):.2f} MB")

    log_file = '/content/processing_log.txt'
    if os.path.exists(log_file):
        os.remove(log_file)
    logging.basicConfig(filename=log_file, level=logging.INFO,
                       format='%(asctime)s - %(message)s')

    cap = cv2.VideoCapture(file_name)
    input_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    input_duration = frame_count / input_fps
    output_fps = input_fps * interpolation_factor
    expected_frame_count = frame_count * interpolation_factor
    cap.release()
    print(f"Input FPS: {input_fps}, Frames: {frame_count}, Duration: {input_duration:.2f} sec")
    print(f"Target FPS: {output_fps}, Expected duration: {input_duration:.2f} sec")

    base_file_name = os.path.basename(file_name)
    output_file_name = base_file_name.rsplit('.', 1)[0] + f'_{interpolation_factor}x.mp4'
    if output_folder == "google_drive":
        save_path = '/content/drive/MyDrive/'
    elif output_folder == "root":
        save_path = '/content/'
    else:
        save_path = '/content/'
    output_video = os.path.join(save_path, output_file_name)
    temp_output_video = os.path.join(save_path, "temp_" + output_file_name)

    model_path = f"/content/FLAVR_{model_type}.pth"
    if not os.path.exists(model_path):
        print(f"Error: model not found at {model_path}. Run the 'Install FLAVR Dependencies' cell.")
    elif not os.path.exists("/content/FLAVR/interpolate.py"):
        print("Error: interpolate.py script not found. Make sure the FLAVR repository is downloaded.")
    else:
        temp_dir = "/content/temp_segments"
        shutil.rmtree(temp_dir, ignore_errors=True)
        os.makedirs(temp_dir, exist_ok=True)

        print(f"Splitting video into segments of {segment_duration} seconds...")
        segment_pattern = os.path.join(temp_dir, "segment_%03d.mp4")
        cmd_split = [
            "ffmpeg", "-i", file_name, "-c", "copy", "-map", "0",
            "-segment_time", str(segment_duration), "-f", "segment",
            "-reset_timestamps", "1", "-y", segment_pattern
        ]
        result_split = subprocess.run(cmd_split, capture_output=True, text=True)
        if result_split.returncode != 0:
            print("Error splitting video:")
            print(result_split.stderr)
            raise subprocess.CalledProcessError(result_split.returncode, cmd_split)

        segments = sorted([f for f in os.listdir(temp_dir) if f.endswith(".mp4") and not f.endswith(f"_{interpolation_factor}x.mp4")])
        print(f"Segments created: {len(segments)}")

        processed_segments = []
        start_time = time.time()
        for i, segment in enumerate(segments):
            segment_path = os.path.join(temp_dir, segment)
            temp_output = f"/content/processed_segment_{i}_{interpolation_factor}x.mp4"
            flavr_output = f"/content/{segment.rsplit('.', 1)[0]}_{interpolation_factor}x.mp4"
            first_frame_video = f"/content/first_frame_{i}.mp4"
            first_frame_dup = f"/content/first_frame_dup_{i}.mp4"
            second_frame_video = f"/content/second_frame_{i}.mp4"
            last_frame_video = f"/content/last_frame_{i}.mp4"
            last_frame_dup = f"/content/last_frame_dup_{i}.mp4"
            final_segment = f"/content/final_segment_{i}_{interpolation_factor}x.mp4"

            for f in [temp_output, flavr_output, first_frame_video, first_frame_dup, second_frame_video, last_frame_video, last_frame_dup, final_segment]:
                if os.path.exists(f):
                    os.remove(f)

            cap = cv2.VideoCapture(segment_path)
            seg_fps = cap.get(cv2.CAP_PROP_FPS)
            seg_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            seg_duration = seg_frame_count / seg_fps
            cap.release()
            seg_output_fps = seg_fps * interpolation_factor
            frame_duration = 1 / seg_output_fps
            expected_seg_frame_count = seg_frame_count * interpolation_factor

            logging.info(f"Segment {i} original: FPS {seg_fps}, Frames {seg_frame_count}, Duration {seg_duration:.2f} sec")
            logging.info(f"Expected frames after interpolation: {expected_seg_frame_count}")

            cmd_extract_first = [
                "ffmpeg", "-i", segment_path, "-frames:v", "1", "-r", str(int(seg_output_fps)),
                "-c:v", "libx264", "-c:a", "aac", "-t", str(frame_duration), "-y", first_frame_video
            ]
            subprocess.run(cmd_extract_first, capture_output=True, text=True)
            shutil.copy(first_frame_video, first_frame_dup)

            cmd_extract_second = [
                "ffmpeg", "-i", segment_path, "-vf", "select='eq(n\,1)'",
                "-frames:v", "1", "-r", str(int(seg_output_fps)), "-c:v", "libx264", "-c:a", "aac",
                "-t", str(frame_duration), "-y", second_frame_video
            ]
            subprocess.run(cmd_extract_second, capture_output=True, text=True)

            cmd_extract_last = [
                "ffmpeg", "-i", segment_path, "-vf", f"select='eq(n\,{seg_frame_count-1})'",
                "-frames:v", "1", "-r", str(int(seg_output_fps)), "-c:v", "libx264", "-c:a", "aac",
                "-t", str(frame_duration), "-y", last_frame_video
            ]
            subprocess.run(cmd_extract_last, capture_output=True, text=True)
            shutil.copy(last_frame_video, last_frame_dup)

            cmd_flavr = [
                "python", "/content/FLAVR/interpolate.py",
                "--input_video", segment_path,
                "--factor", str(interpolation_factor),
                "--load_model", model_path,
                "--output_fps", str(int(seg_output_fps))
            ]

            elapsed_time = time.time() - start_time
            progress = (i + 1) / len(segments)
            if elapsed_time > 0 and progress > 0:
                estimated_total = elapsed_time / progress
                remaining_time = estimated_total - elapsed_time
            else:
                estimated_total = 0
                remaining_time = 0

            bar_length = 30
            filled = int(bar_length * progress)
            bar = '=' * filled + '-' * (bar_length - filled)
            elapsed_str = f"{int(elapsed_time // 60):02d}:{int(elapsed_time % 60):02d}"
            remaining_str = f"{int(remaining_time // 60):02d}:{int(remaining_time % 60):02d}"
            sys.stdout.write(f"\rProcessing: [{bar}] {progress:.1%} | Elapsed: {elapsed_str} | Remaining: {remaining_str} ({i+1}/{len(segments)})")
            sys.stdout.flush()

            result_flavr = subprocess.run(cmd_flavr, capture_output=True, text=True)
            if result_flavr.returncode != 0:
                print(f"\nError processing segment {segment}:")
                print(result_flavr.stderr)
                raise subprocess.CalledProcessError(result_flavr.returncode, cmd_flavr)
            elif not os.path.exists(flavr_output):
                print(f"\nError: FLAVR output file not created: {flavr_output}")
                raise FileNotFoundError(f"FLAVR output not created: {flavr_output}")
            else:
                shutil.move(flavr_output, temp_output)

                cap = cv2.VideoCapture(temp_output)
                flavr_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                flavr_duration = flavr_frame_count / seg_output_fps
                cap.release()
                logging.info(f"Segment {i} after FLAVR: FPS {seg_output_fps}, Frames {flavr_frame_count}, Duration {flavr_duration:.2f} sec")

                concat_list_temp = f"/content/concat_list_{i}.txt"
                with open(concat_list_temp, "w") as f:
                    f.write(f"file '{first_frame_video}'\n")
                    f.write(f"file '{first_frame_dup}'\n")
                    f.write(f"file '{second_frame_video}'\n")
                    f.write(f"file '{temp_output}'\n")
                    f.write(f"file '{last_frame_dup}'\n")
                    f.write(f"file '{last_frame_video}'\n")

                cmd_concat_segment = [
                    "ffmpeg", "-f", "concat", "-safe", "0", "-i", concat_list_temp,
                    "-c:v", "libx264", "-c:a", "aac", "-r", str(int(seg_output_fps)), "-y", final_segment
                ]
                result_concat_seg = subprocess.run(cmd_concat_segment, capture_output=True, text=True)
                if result_concat_seg.returncode != 0:
                    print(f"\nError concatenating segment {segment}:")
                    print(result_concat_seg.stderr)
                    raise subprocess.CalledProcessError(result_concat_seg.returncode, cmd_concat_segment)

                cap = cv2.VideoCapture(final_segment)
                final_seg_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                final_seg_duration = final_seg_frame_count / seg_output_fps
                cap.release()
                logging.info(f"Segment {i} final: FPS {seg_output_fps}, Frames {final_seg_frame_count}, Duration {final_seg_duration:.2f} sec")
                logging.info(f"Frame difference (final - expected): {final_seg_frame_count - expected_seg_frame_count}")
                logging.info(f"Duration difference (final - original): {final_seg_duration - seg_duration:.2f} sec")
                processed_segments.append(final_segment)

                for f in [temp_output, first_frame_video, first_frame_dup, second_frame_video, last_frame_video, last_frame_dup, concat_list_temp]:
                    if os.path.exists(f):
                        os.remove(f)

            gc.collect()

        print()

        cmd_bitrate = ["ffprobe", "-v", "error", "-show_entries", "format=bit_rate", "-of", "default=noprint_wrappers=1:nokey=1", file_name]
        result_bitrate = subprocess.run(cmd_bitrate, capture_output=True, text=True)
        input_bitrate = int(result_bitrate.stdout.strip()) if result_bitrate.stdout.strip().isdigit() else 2000000
        output_bitrate = input_bitrate * interpolation_factor
        print(f"Original bitrate: {input_bitrate / 1000:.0f} kbps, Target bitrate: {output_bitrate / 1000:.0f} kbps")

        print("Merging processed segments...")
        concat_list = os.path.join(temp_dir, "concat_list.txt")
        with open(concat_list, "w") as f:
            for seg in processed_segments:
                f.write(f"file '{seg}'\n")

        cmd_concat = [
            "ffmpeg", "-f", "concat", "-safe", "0", "-i", concat_list,
            "-c:v", "libx264", "-r", str(int(output_fps)),
            "-b:v", f"{output_bitrate}", "-an", "-sn", "-y", temp_output_video
        ]
        result_concat = subprocess.run(cmd_concat, capture_output=True, text=True)
        if result_concat.returncode != 0:
            print("Error merging segments:")
            print(result_concat.stderr)
            raise subprocess.CalledProcessError(result_concat.returncode, cmd_concat)

        cmd_final = [
            "ffmpeg", "-i", temp_output_video, "-i", file_name,
            "-map", "0:v", "-map", "1:a?", "-map", "1:s?",
            "-c:v", "copy", "-c:a", "copy", "-c:s", "copy",
            "-r", str(int(output_fps)), "-y", output_video
        ]
        result_final = subprocess.run(cmd_final, capture_output=True, text=True)
        if result_final.returncode != 0:
            print("Error adding audio and subtitles:")
            print(result_final.stderr)
            raise subprocess.CalledProcessError(result_final.returncode, cmd_final)

        cap = cv2.VideoCapture(output_video)
        final_frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        final_duration = final_frame_count / output_fps
        cap.release()
        print(f"Final video: FPS: {output_fps}, Frames: {final_frame_count}, Duration: {final_duration:.2f} sec")
        print(f"Original video duration: {input_duration:.2f} sec")
        print(f"Duration difference (final - original): {final_duration - input_duration:.2f} sec")

        shutil.rmtree(temp_dir)
        for seg in processed_segments:
            if os.path.exists(seg):
                os.remove(seg)
        if os.path.exists(temp_output_video):
            os.remove(temp_output_video)

        print(f"\nFPS-enhanced video saved: {output_video}")
        print(f"Additional logs saved to: {log_file}")
        print("Use the next cell to download or check the file manually.")

In [ ]:
#@title ##**Compare videos (optional)** { display-mode: "form" }
from IPython.display import display, HTML
import os
import base64

original_video_path = file_name
processed_video_path = output_video

if not os.path.exists(original_video_path):
    raise ValueError(f"Оригинальное видео не найдено по пути: {original_video_path}")
if not os.path.exists(processed_video_path):
    raise ValueError(f"Обработанное видео не найдено по пути: {processed_video_path}")

original_size = os.path.getsize(original_video_path) / (1024 * 1024)
processed_size = os.path.getsize(processed_video_path) / (1024 * 1024)
print(f"Размер оригинального видео: {original_size:.2f} МБ")
print(f"Размер обработанного видео: {processed_size:.2f} МБ")


def video_to_base64(video_path):
    with open(video_path, "rb") as video_file:
        video_data = video_file.read()
    return base64.b64encode(video_data).decode('utf-8')

original_base64 = video_to_base64(original_video_path)
processed_base64 = video_to_base64(processed_video_path)

html_code = f"""
<div style="display: flex; justify-content: center; flex-direction: column; align-items: center;">
    <div style="display: flex; justify-content: center;">
        <div style="margin-right: 10px;">
            <video id="originalVideo" width="400" controls preload="auto">
                <source src="data:video/mp4;base64,{original_base64}" type="video/mp4">
                Ваш браузер не поддерживает видео.
            </video>
            <p>Оригинальное видео</p>
        </div>
        <div>
            <video id="processedVideo" width="400" controls preload="auto">
                <source src="data:video/mp4;base64,{processed_base64}" type="video/mp4">
                Ваш браузер не поддерживает видео.
            </video>
            <p>Обработанное видео</p>
        </div>
    </div>
    <button id="playPauseBtn" style="margin-top: 10px; padding: 10px 20px; font-size: 16px;">Play</button>
</div>
<script>
(function() {{
    var originalVideo = document.getElementById("originalVideo");
    var processedVideo = document.getElementById("processedVideo");
    var playPauseBtn = document.getElementById("playPauseBtn");
    var isPlaying = false;

    playPauseBtn.disabled = false;

    function playBoth() {{
        Promise.all([
            originalVideo.play().catch(function(error) {{
                console.log("Ошибка воспроизведения оригинального видео:", error);
            }}),
            processedVideo.play().catch(function(error) {{
                console.log("Ошибка воспроизведения обработанного видео:", error);
            }})
        ]).then(function() {{
            playPauseBtn.textContent = "Pause";
            isPlaying = true;
        }}).catch(function(error) {{
            console.log("Не удалось воспроизвести видео:", error);
        }});
    }}

    function pauseBoth() {{
        originalVideo.pause();
        processedVideo.pause();
        playPauseBtn.textContent = "Play";
        isPlaying = false;
    }}

    playPauseBtn.addEventListener("click", function() {{
        if (isPlaying) {{
            pauseBoth();
        }} else {{
            playBoth();
        }}
    }});

    originalVideo.addEventListener("play", function() {{
        if (processedVideo.paused) processedVideo.play();
        playPauseBtn.textContent = "Pause";
        isPlaying = true;
    }});
    processedVideo.addEventListener("play", function() {{
        if (originalVideo.paused) originalVideo.play();
        playPauseBtn.textContent = "Pause";
        isPlaying = true;
    }});
    originalVideo.addEventListener("pause", function() {{
        if (!processedVideo.paused) processedVideo.pause();
        playPauseBtn.textContent = "Play";
        isPlaying = false;
    }});
    processedVideo.addEventListener("pause", function() {{
        if (!originalVideo.paused) originalVideo.pause();
        playPauseBtn.textContent = "Play";
        isPlaying = false;
    }});

    originalVideo.addEventListener("timeupdate", function() {{
        if (Math.abs(originalVideo.currentTime - processedVideo.currentTime) > 0.5) {{
            processedVideo.currentTime = originalVideo.currentTime;
        }}
    }});
    processedVideo.addEventListener("timeupdate", function() {{
        if (Math.abs(processedVideo.currentTime - originalVideo.currentTime) > 0.5) {{
            originalVideo.currentTime = processedVideo.currentTime;
        }}
    }});

    console.log("Скрипт синхронизации видео инициализирован");
}})();
</script>
"""

display(HTML(html_code))

In [ ]:
#@title ##**Compare frames (optional)** { display-mode: "form" }
import cv2
import os
import numpy as np
from PIL import Image as PILImage
from IPython.display import display, Image

max_frames_to_show = 3 #@param {type:"slider", min:1, max:50, step:1}

if 'file_name' not in globals() or file_name is None or not os.path.exists(file_name):
    print("Error: Original video file is not selected or does not exist.")
elif 'output_video' not in globals() or not os.path.exists(output_video):
    print("Error: Interpolated video file not found. Run the 'Run' cell first.")
else:
    cap_orig = cv2.VideoCapture(file_name)
    cap_interp = cv2.VideoCapture(output_video)

    if not cap_orig.isOpened() or not cap_interp.isOpened():
        print("Error: Could not open one or both video files.")
    else:
        print(f"Building frame comparison strip for original ({file_name}) and interpolated ({output_video}) videos:")
        print(f"Interpolation factor: {interpolation_factor}")

        orig_frame_count = int(cap_orig.get(cv2.CAP_PROP_FRAME_COUNT))
        interp_frame_count = int(cap_interp.get(cv2.CAP_PROP_FRAME_COUNT))

        max_orig_frames = min(max_frames_to_show, orig_frame_count)
        max_interp_frames = min(max_orig_frames * interpolation_factor, interp_frame_count)

        frame_width, frame_height = 640, 360

        orig_frames = []
        interp_frames = []

        from PIL import ImageDraw, ImageFont
        try:
            font = ImageFont.truetype("DejaVuSans.ttf", 30)
        except:
            font = ImageFont.load_default()

        for orig_frame_num in range(max_orig_frames):
            ret_orig, frame_orig = cap_orig.read()
            if not ret_orig:
                print(f"Error reading original frame {orig_frame_num}")
                break
            frame_orig_rgb = cv2.cvtColor(frame_orig, cv2.COLOR_BGR2RGB)
            pil_img = PILImage.fromarray(frame_orig_rgb).resize((frame_width, frame_height))
            draw = ImageDraw.Draw(pil_img)
            draw.text((10, frame_height-40), f"#{orig_frame_num}", fill="white", font=font)
            orig_frames.append(pil_img)

        for interp_frame_num in range(max_interp_frames):
            ret_interp, frame_interp = cap_interp.read()
            if not ret_interp:
                print(f"Error reading interpolated frame {interp_frame_num}")
                break
            frame_interp_rgb = cv2.cvtColor(frame_interp, cv2.COLOR_BGR2RGB)
            pil_img = PILImage.fromarray(frame_interp_rgb).resize((frame_width, frame_height))
            draw = ImageDraw.Draw(pil_img)
            draw.text((10, frame_height-40), f"#{interp_frame_num}", fill="white", font=font)
            interp_frames.append(pil_img)

        cap_orig.release()
        cap_interp.release()

        blank_frame = PILImage.new('RGB', (frame_width, frame_height), color='black')
        draw_blank = ImageDraw.Draw(blank_frame)
        draw_blank.text((10, frame_height-40), "#N/A", fill="white", font=font)

        orig_strip_frames = []
        for i in range(max_interp_frames):
            if i % interpolation_factor == 0 and i // interpolation_factor < len(orig_frames):
                orig_strip_frames.append(orig_frames[i // interpolation_factor])
            else:
                orig_strip_frames.append(blank_frame.copy())

        try:
            num_frames = min(len(orig_strip_frames), len(interp_frames))

            orig_strip = PILImage.new('RGB', (frame_width * num_frames, frame_height))
            interp_strip = PILImage.new('RGB', (frame_width * num_frames, frame_height))

            for i in range(num_frames):
                x_offset = i * frame_width
                if i < len(orig_strip_frames):
                    orig_strip.paste(orig_strip_frames[i], (x_offset, 0))
                if i < len(interp_frames):
                    interp_strip.paste(interp_frames[i], (x_offset, 0))

            comparison_strip = PILImage.new('RGB', (frame_width * num_frames, frame_height * 2))
            comparison_strip.paste(orig_strip, (0, 0))
            comparison_strip.paste(interp_strip, (0, frame_height))

            draw = ImageDraw.Draw(comparison_strip)
            try:
                font = ImageFont.truetype("DejaVuSans.ttf", 20)
            except:
                font = ImageFont.load_default()

            draw.text((10, 10), "Original video", fill="white", font=font)
            draw.text((10, frame_height + 10), "Interpolated video", fill="white", font=font)

            strip_path = "/content/comparison_strip.jpg"
            comparison_strip.save(strip_path)

            print("\nOriginal frames (top) vs Interpolated frames (bottom):")
            display(Image(filename=strip_path))

        except Exception as e:
            print(f"Error creating comparison image: {str(e)}")

        print("\nFrame comparison strip completed.")

In [ ]:
#@title ##**Download** { display-mode: "form" }
import os
from google.colab import files

if os.path.exists(output_video):
    print(f"Stabilized video saved at: {output_video}")
    files.download(output_video)
else:
    print("Error: Stabilized video was not created.")
